In [1]:
import os
import sys
import pandas as pd 
import numpy as np 
import scipy.sparse as ssparse 
import scipy.io as sio 

sys.path.insert(0, '/Users/bingyu')
from sp import interface 

In [2]:
nodes_df0 = pd.read_csv('../projects/berkeley/network_inputs/osm_nodes.csv')
links_df0 = pd.read_csv('../projects/berkeley/network_inputs/osm_edges.csv')
# links_df0 = pd.read_csv('../projects/berkeley/network_inputs/modified_network_edges.csv')
wgh = links_df0['length']
row = links_df0['start_igraph']
col = links_df0['end_igraph']
assert max(np.max(row)+1, np.max(col)+1) == nodes_df0.shape[0], 'nodes and links dimension do not match, row {}, col {}, nodes {}'.format(np.max(row), np.max(col), nodes_df0.shape[0])
g_coo = ssparse.coo_matrix((wgh, (row, col)), shape=(nodes_df0.shape[0], nodes_df0.shape[0]))
print("({}, {}), {}".format(g_coo.shape[0], g_coo.shape[1], len(g_coo.data)))
sio.mmwrite('network_sparse.mtx', g_coo)
# g_coo = sio.mmread(absolute_path+'/outputs/network_sparse.mtx'.format(folder))
g = interface.readgraph(bytes('network_sparse.mtx', encoding='utf-8'))

(15294, 15294), 37951


In [3]:
### marin: 10185, 2598
### spruce: 2160, 1626
### shasta: 5525, 1623
### oxfful: 14299, 9242
### rose: 4864, 14299
### cedar: 15068, 14483
counterflow_start_igraph, counterflow_end_igraph = 5525, 1623 ### node_id_igraph
counterflow_start_sp = counterflow_start_igraph + 1
counterflow_end_sp = counterflow_end_igraph + 1

### along counterflow
sp_1 = g.dijkstra(counterflow_start_sp, counterflow_end_sp)
sp_dist_1 = sp_1.distance(counterflow_end_sp)
print(sp_dist_1)
sp_route_1 = sp_1.route(counterflow_end_sp)
path_1 = [(start_sp-1, end_sp-1) for (start_sp, end_sp) in sp_route_1]
print(path_1)
path_df_1 = pd.DataFrame(path_1, columns=['start_igraph', 'end_igraph'])
path_df_1['along'] = 1

### against counterflow
sp_2 = g.dijkstra(counterflow_end_sp, counterflow_start_sp)
sp_dist_2 = sp_2.distance(counterflow_start_sp)
print(sp_dist_2)
sp_route_2 = sp_2.route(counterflow_start_sp)
path_2 = [(start_sp-1, end_sp-1) for (start_sp, end_sp) in sp_route_2]
print(path_2)
path_df_2 = pd.DataFrame(path_2, columns=['start_igraph', 'end_igraph'])
path_df_2['along'] = 0

counterflow_route = pd.concat([path_df_1, path_df_2])
counterflow_route = pd.merge(counterflow_route, links_df0[['start_igraph', 'end_igraph', 'edge_id_igraph','geometry']], how='left', on=['start_igraph', 'end_igraph'])
counterflow_route.to_csv('shasta.csv', index=False)

2879.467138019012
[(5525, 1543), (1543, 1186), (1186, 3978), (3978, 3977), (3977, 4558), (4558, 4377), (4377, 1228), (1228, 6428), (6428, 1231), (1231, 1232), (1232, 5864), (5864, 5863), (5863, 3477), (3477, 15068), (15068, 5267), (5267, 3505), (3505, 10685), (10685, 13193), (13193, 7626), (7626, 1626), (1626, 1623)]
2889.4526853461275
[(1623, 1626), (1626, 7626), (7626, 13193), (13193, 10685), (10685, 3505), (3505, 5267), (5267, 15068), (15068, 3477), (3477, 5863), (5863, 1234), (1234, 5864), (5864, 1232), (1232, 1231), (1231, 6428), (6428, 1228), (1228, 4377), (4377, 4558), (4558, 3977), (3977, 3978), (3978, 1186), (1186, 1543), (1543, 5525)]
